In [1]:
pip install rapidfuzz lifetimes matplotlib tqdm pytz

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
from rapidfuzz import process, fuzz
from tqdm import tqdm
import pytz
from lifetimes import GammaGammaFitter

import warnings
warnings.filterwarnings('ignore')

In [2]:
online_content_data = pd.read_csv('Data/XYZ_NETWORKS_ONLINE_CONTENT_CONSUMPTION.csv')
online_content_data.head()

,ID,URL,DATE,SCORE
0,028ZIWM575,https://www.vodmedia.com/target-locked-episode-10,2023-05-23T17:26:54+00:00,0.44772
1,862RYTF544,https://www.videodirect.com/stormfront-the-fin...,2022-03-03T17:32:02+00:00,NaN
2,467WUAP768,https://www.vodmedia.com/deep-waters-submarine...,2023-08-04T08:57:06+00:00,0.66931
3,765KZVX773,https://www.watchnow.com/deep-waters-submarine...,2023-01-25T12:17:42+00:00,0.88933
4,774ZQUY606,https://www.vodflix.com/target-locked-s2e5,2023-08-03T02:05:50+00:00,0.64074


In [3]:
online_content_data.shape

(799895, 4)

## Data Preprocessing

In [4]:
# drop rows with na
df = online_content_data.dropna()

In [5]:
def extract_show(url):
    path = re.sub(r'^https?://www\.[^/]+/', '', url)
    path = re.sub(r'^(movies/|shows/|series/)', '', path)
    path = re.sub(r'(-episode.*|-season.*|-s\d+e\d+.*|-part.*)$', '', path)
    show = path.replace('-', ' ').strip()
    return show if show else None

In [6]:
# append brand and show name column to df
df['Show'] = df['URL'].apply(extract_show)

In [7]:
# transform DATE to datetime object
df['DATE'] = pd.to_datetime(df['DATE'])

In [8]:
df['year'] = df['DATE'].dt.year
df['month'] = df['DATE'].dt.month
df['day'] = df['DATE'].dt.day
df['hour'] = df['DATE'].dt.hour
df['dayofweek'] = df['DATE'].dt.dayofweek+1

In [9]:
# filter out rows where 'SCORE' < 0.5

df = df[df['SCORE'] > 0.2]

In [10]:
df.head()

,ID,URL,DATE,SCORE,Show,year,month,day,hour,dayofweek
0,028ZIWM575,https://www.vodmedia.com/target-locked-episode-10,2023-05-23 17:26:54+00:00,0.44772,target locked,2023,5,23,17,2
2,467WUAP768,https://www.vodmedia.com/deep-waters-submarine...,2023-08-04 08:57:06+00:00,0.66931,deep waters submarine showdown,2023,8,4,8,5
3,765KZVX773,https://www.watchnow.com/deep-waters-submarine...,2023-01-25 12:17:42+00:00,0.88933,deep waters submarine showdown,2023,1,25,12,3
4,774ZQUY606,https://www.vodflix.com/target-locked-s2e5,2023-08-03 02:05:50+00:00,0.64074,target locked,2023,8,3,2,4
5,234FCFH350,https://www.vodmedia.com/the-last-outpost-s4e9,2023-02-05 15:24:11+00:00,0.86975,the last outpost,2023,2,5,15,7


In [11]:
df.shape

(467708, 10)

In [12]:
# map brand to each show

brand_shows = {
    'LimeLight': [
        'Fashion Frenzy: The Runway Wars',
        'Home Makeover Rescue',
        'Celebrity Chefs Showdown',
        'Style Seekers: Global Trends',
        'The Perfect Party Planner',
        'Behind the Glam: Celebrity Secrets',
        'Living Large: Luxury Homes Edition',
        'Fitness Gurus: Transformations',
        'Destination Wedding Dreams',
        'Extreme Makeovers: House Flips'
    ],
    'Pulse': [
        'Edge of Extinction',
        'Highway Heist',
        'Stormfront: The Final Mission',
        'Deep Waters: Submarine Showdown',
        'Midnight Pursuit',
        'Target Locked',
        'Chasing Shadows: Elite Unit',
        'Blood Oath: Vigilante Justice',
        'Flight Risk',
        'The Last Outpost'
    ],
    'ChillStream': [
        'Oceans Untamed: The Deep Unknown',
        'Ancient Civilizations Uncovered',
        'World Wonders: Nature\'s Marvels',
        'The Amazon Trail: A Journey Through the Rainforest',
        'Surviving Antarctica: The Final Frontier',
        'Wildlife Warriors: Protectors of the Endangered',
        'Unseen Worlds: Microscopic Marvels',
        'Nomads of the North: Life in the Arctic',
        'Mount Everest: Beyond the Summit',
        'Wonders of the Coral Reef'
    ],
    'RetroReel': [
        'Vintage Noir: Tales of the 50s',
        'Hollywood Gold: The Best of the Silver Screen',
        'The Golden Age of Television',
        'Classic Cartoons Rewind',
        'Heroes of the West: Old Cowboy Adventures',
        'The Big Picture: Cinema’s Finest Years',
        'Love in Black and White: Romantic Classics',
        'Retro Sci-Fi: Space and Beyond',
        'Timeless Thrillers: Hitchcock’s Legacy',
        '1950s Sitcom Showcase'
    ],
    'CineQuest': [
        'The Forgotten Kingdom',
        'Unwritten Laws',
        'Echoes of the Past',
        'Shattered Mirror: A Tale of Betrayal',
        'Lies Beneath the Surface',
        'After the Storm: A Family’s Battle',
        'In the Shadow of Giants',
        'Fallen Crown: The Battle for the Throne',
        'Last Sunset: A Dystopian Love Story',
        'Whispers in the Dark'
    ],
    'Adrenaline': [
        'Chopper Chase: Ride to Freedom',
        'Survivalist: Desert Run',
        'The High Stakes Heist',
        'Crash Course: Daredevil Driving',
        'Outrun the Law: Speed Trials',
        'Extreme Elevation: Mountain Mayhem',
        'Race Against Time: Urban Parkour',
        'Rogue Waves: Ocean Survival',
        'Battle on the Bridge',
        'Canyon Plunge: The Ultimate Cliff Dive'
    ],
    'DarkMatter': [
        'Mind Games: The Final Puzzle',
        'The Silent Watcher',
        'Vanishing Point: The Missing Files',
        'Twisted Shadows',
        'Behind Closed Doors: A Psychological Nightmare',
        'Double Crossed: The Deep Conspiracy',
        'The Last Witness',
        'False Alibis',
        'The Confession That Never Was',
        'Voices in the Dark'
    ],
    'SilverScreen Classics': [
        'Midnight Masquerade',
        'The Gunslinger’s Code',
        'Dancing in the Rain: A Musical Romance',
        'Secrets of the Starlet',
        'Heartstrings: A Love to Remember',
        'Man on the Moon: A Sci-Fi Classic',
        'Western Trails: The Lone Ranger',
        'Whispers in the Night',
        'The Jazz Club: Music and Romance',
        'Beneath the Scarlet Veil'
    ],
    'TimeCapsule TV': [
        'The Brady Family Reunion',
        'Golden Laughs: Classic Sitcoms',
        'Super Sleuth: Detective Rewind',
        'Game Show Fever: Retro Revival',
        'The Wonder Years: Growing Up in the 80s',
        'The Mid-Century Mystery Hour',
        'Our Neighborhood: Family Stories from the 70s',
        'Classic Comedy Club',
        'The Great Quiz Show Showdown',
        'The Retro Detective: Case Files Reopened'
    ],
    'TasteMakers': [
        'The Chef Showdown: Battle of the Kitchens',
        'Food Truck Frenzy',
        'Global Gourmet: Culinary Journeys',
        'Bake Off Bonanza',
        'Flavors of the World: Street Eats',
        'Top Chef All-Stars: The Final Feast',
        'Sugar Rush: Desserts on Deadline',
        'Master Butcher: Carving the Perfect Cut',
        'Spice Wars: The Heat is On',
        'Farm to Table: The Chef’s Challenge'
    ],
    'DesignLab': [
        'Renovation Rescue: Extreme Makeovers',
        'Modern Marvels: Futuristic Home Designs',
        'Tiny Homes, Big Dreams',
        'Design Masters: Showdown of the Styles',
        'Budget Renovation: Luxe for Less',
        'Curb Appeal: Outdoor Transformations',
        'Dream Spaces: Designer’s Pick',
        'Flip or Flop: The Ultimate Remodel',
        'Eco-Living: Sustainable Homes',
        'Minimalist Dreams: Designing with Less'
    ],
    'PopCulture Now': [
        'Star Style: Hollywood’s Fashion Icons',
        'The Influencer Effect: Social Media Superstars',
        'Celebrity Gossip Confidential',
        'Red Carpet Rundown: Awards Show Style',
        'Hot List: Who’s Trending Now?',
        'Fashion Faux Pas: Hits and Misses',
        'Inside the A-List: Celebrity Homes',
        'The Insta-Celeb Diaries',
        'Behind the Fame: The Rise of Pop Culture Icons',
        'Entertainment Flash: The Week in Review'
    ]
}

In [13]:
# show to brand mapping

show_to_brand = {}
for brand, shows in brand_shows.items():
    for show in shows:
        show_to_brand[show.lower()] = brand

In [14]:
all_show_names = list(show_to_brand.keys())

In [15]:
# fuzzy match function

def match_show_name(show_name, choices, scorer=fuzz.WRatio, cutoff=80):
    """
    Matches the show_name to the closest show in choices using fuzzy matching.
    Returns the matched show name if the score is above the cutoff; otherwise, returns None.
    """
    if pd.isnull(show_name):
        return None
    show_name = show_name.lower()
    match = process.extractOne(show_name, choices, scorer=scorer)
    if match and match[1] >= cutoff:
        return match[0]
    else:
        return None

In [16]:
tqdm.pandas()

df['Matched_Show'] = df['Show'].progress_apply(lambda x: match_show_name(x, all_show_names))

100%|██████████| 467708/467708 [01:06<00:00, 7081.87it/s] 


In [17]:
df[df['Matched_Show'].isna()]

,ID,URL,DATE,SCORE,Show,year,month,day,hour,dayofweek,Matched_Show
22431,725XTDH624,Here are 100 more URLs following your specifie...,2022-07-22 12:12:22+00:00,0.26150,Here are 100 more URLs following your specifie...,2022,7,22,12,5,None
22861,321DQGU207,Here are 100 more URLs following your specifie...,2022-08-26 23:17:19+00:00,0.72855,Here are 100 more URLs following your specifie...,2022,8,26,23,5,None
22887,859GXFC303,Here are 100 more URLs following your specifie...,2022-11-11 09:29:24+00:00,0.80164,Here are 100 more URLs following your specifie...,2022,11,11,9,5,None
22946,033OCLA420,Here are 100 more URLs following your specifie...,2022-03-26 05:35:49+00:00,0.27399,Here are 100 more URLs following your specifie...,2022,3,26,5,6,None
23499,454DREA418,Here are 100 more URLs following your specifie...,2022-11-25 17:21:14+00:00,0.54828,Here are 100 more URLs following your specifie...,2022,11,25,17,5,None
...,...,...,...,...,...,...,...,...,...,...,...
770377,965OZYV564,Here are 100 more URLs following your specifie...,2023-01-04 20:56:34+00:00,0.95549,Here are 100 more URLs following your specifie...,2023,1,4,20,3,None
770670,989XJRY624,Here are 100 more URLs following your specifie...,2023-03-14 19:21:24+00:00,0.22902,Here are 100 more URLs following your specifie...,2023,3,14,19,2,None
770844,796FGZK586,Here are 100 more URLs following your specifie...,2022-05-16 04:01:01+00:00,0.26654,Here are 100 more URLs following your specifie...,2022,5,16,4,1,None
770867,687VVTG679,Here are 100 more URLs following your specifie...,2023-07-26 19:50:18+00:00,0.84241,Here are 100 more URLs following your specifie...,2023,7,26,19,3,None


In [18]:
# drop invalid URL format rows

df = df[~df['Matched_Show'].isna()]

In [19]:
# map each row to a brand

df['BRAND'] = df['Matched_Show'].map(show_to_brand)

In [20]:
df['BRAND'].isna().sum()

np.int64(0)

In [21]:
df.head()

,ID,URL,DATE,SCORE,Show,year,month,day,hour,dayofweek,Matched_Show,BRAND
0,028ZIWM575,https://www.vodmedia.com/target-locked-episode-10,2023-05-23 17:26:54+00:00,0.44772,target locked,2023,5,23,17,2,target locked,Pulse
2,467WUAP768,https://www.vodmedia.com/deep-waters-submarine...,2023-08-04 08:57:06+00:00,0.66931,deep waters submarine showdown,2023,8,4,8,5,deep waters: submarine showdown,Pulse
3,765KZVX773,https://www.watchnow.com/deep-waters-submarine...,2023-01-25 12:17:42+00:00,0.88933,deep waters submarine showdown,2023,1,25,12,3,deep waters: submarine showdown,Pulse
4,774ZQUY606,https://www.vodflix.com/target-locked-s2e5,2023-08-03 02:05:50+00:00,0.64074,target locked,2023,8,3,2,4,target locked,Pulse
5,234FCFH350,https://www.vodmedia.com/the-last-outpost-s4e9,2023-02-05 15:24:11+00:00,0.86975,the last outpost,2023,2,5,15,7,the last outpost,Pulse


In [22]:
# count of user visiting each brand

df.groupby('BRAND')['ID'].nunique().sort_values(ascending=False)

BRAND
LimeLight                18929
SilverScreen Classics    13502
TimeCapsule TV           13501
DarkMatter               13030
Adrenaline               12891
PopCulture Now           12818
DesignLab                12809
TasteMakers              12710
RetroReel                12132
Pulse                    10854
CineQuest                 9884
ChillStream               8513
Name: ID, dtype: int64

In [23]:
df.groupby('BRAND')['SCORE'].mean().sort_values(ascending=False)

BRAND
TasteMakers              0.601952
DesignLab                0.601341
PopCulture Now           0.601105
Pulse                    0.600751
ChillStream              0.600422
CineQuest                0.600260
RetroReel                0.599981
SilverScreen Classics    0.599219
TimeCapsule TV           0.599005
Adrenaline               0.598636
LimeLight                0.598481
DarkMatter               0.597958
Name: SCORE, dtype: float64

## Train Test Split

train:   X (2022-01 to 2022-12)  y (2023-01 to 2023-06)

test:    X (2022-06 to 2023-06)  y (2023-06 to 2023-12)

predict: X (2023-01 to 2023-12)  y (2024-01 to 2024-06)

In [24]:
train_start, train_end = '2022-01-01', '2022-12-31'
test_start, test_end = '2022-06-01', '2023-06-30'
predict_start, predict_end = '2023-01-01', '2023-12-31'

In [25]:
# Train set: X from 2022-01 to 2022-12, y from 2023-01 to 2023-06
train_X = df[(df['DATE'] >= train_start) & (df['DATE'] <= train_end)]
train_y = df[(df['DATE'] > '2022-12-31') & (df['DATE'] <= '2023-06-30')]

In [26]:
# Test set: X from 2022-06 to 2023-06, y from 2023-06 to 2023-12
test_X = df[(df['DATE'] >= test_start) & (df['DATE'] <= test_end)]
test_y = df[(df['DATE'] > '2023-06-30') & (df['DATE'] <= '2023-12-31')]

In [27]:
# Prediction set: X from 2023-01 to 2023-12, y from 2024-01 to 2024-06
predict_X = df[(df['DATE'] >= predict_start) & (df['DATE'] <= predict_end)]
predict_y = None # future data: 6 months

In [28]:
print("Train X shape:", train_X.shape)
print("Train y shape:", train_y.shape)
print("Test X shape:", test_X.shape)
print("Test y shape:", test_y.shape)
print("Predict X shape:", predict_X.shape)

Train X shape: (233412, 12)
Train y shape: (115385, 12)
Test X shape: (251593, 12)
Test y shape: (117698, 12)
Predict X shape: (232459, 12)


## Calculate Recency, Frequency, and Tenure

In [29]:
train_X

,ID,URL,DATE,SCORE,Show,year,month,day,hour,dayofweek,Matched_Show,BRAND
6,796KJIH289,https://www.streammaster.com/movies/highway-he...,2022-04-06 17:55:41+00:00,0.92823,highway heist,2022,4,6,17,3,highway heist,Pulse
9,889FTGK131,https://www.videomaster.com/edge-of-extinction...,2022-06-30 19:32:43+00:00,0.47265,edge of extinction,2022,6,30,19,4,edge of extinction,Pulse
10,521JMTP981,https://www.streammaster.com/movies/highway-he...,2022-04-03 11:37:29+00:00,0.51875,highway heist,2022,4,3,11,7,highway heist,Pulse
14,340ETJA590,https://www.watchtracker.com/highway-heist-epi...,2022-04-22 01:27:56+00:00,0.89393,highway heist,2022,4,22,1,5,highway heist,Pulse
17,876YRMW901,https://www.vodmedia.com/midnight-pursuit-epis...,2022-08-24 07:36:29+00:00,0.42176,midnight pursuit,2022,8,24,7,3,midnight pursuit,Pulse
...,...,...,...,...,...,...,...,...,...,...,...,...
798301,669TZMP098,https://www.videohub.com/surviving-antarctica-...,2022-08-18 08:14:45+00:00,0.89078,surviving antarctica the final frontier,2022,8,18,8,4,surviving antarctica: the final frontier,ChillStream
799590,846PNHQ035,https://www.watchportal.com/mount-everest-beyo...,2022-04-22 02:10:29+00:00,0.75197,mount everest beyond the summit,2022,4,22,2,5,mount everest: beyond the summit,ChillStream
799660,687UUOX756,https://www.videosource.com/the-amazon-trail-a...,2022-08-27 23:47:44+00:00,0.48317,the amazon trail a journey through the rainforest,2022,8,27,23,6,the amazon trail: a journey through the rainfo...,ChillStream
799866,276AVTH635,https://www.videomaster.com/ancient-civilizati...,2022-09-19 12:19:12+00:00,0.76239,ancient civilizations uncovered,2022,9,19,12,1,ancient civilizations uncovered,ChillStream


In [30]:
import pandas as pd

def calculate_rf_scores(df):
    # Aggregate by 'ID' and 'BRAND' to calculate necessary metrics
    df_grouped = df.groupby(['ID', 'BRAND']).agg({
        'SCORE': 'sum',  # Sum of scores to represent weighted frequency
        'DATE': ['min', 'max', 'count']
    }).reset_index()

    # Flatten column names
    df_grouped.columns = ['ID', 'BRAND', 'score_sum', 'first_visit_date', 'last_visit_date', 'visit_count']

    # Calculate Weighted Frequency using score_sum
    df_grouped['frequency'] = df_grouped['score_sum'].round().astype(int)

    # Calculate Recency: time since the customer's last visit
    reference_date = df['DATE'].max()
    df_grouped['recency'] = (reference_date - df_grouped['last_visit_date']).dt.days

    # Calculate Tenure: time between the first visit and the end of the observation period
    df_grouped['tenure'] = (reference_date - df_grouped['first_visit_date']).dt.days

    # Select and return only the relevant columns
    return df_grouped[['ID', 'BRAND', 'recency', 'frequency', 'tenure']]

In [ ]:
# def calculate_frequency_tenure(df):
#     df_grouped = df.groupby(['ID', 'BRAND']).agg({
#         'SCORE': 'sum',
#         'DATE': ['min', 'max']
#     }).reset_index()

#     df_grouped.columns = ['ID', 'BRAND', 'score_sum', 'first_visit_date', 'last_visit_date']

#     # Calculate Frequency: total number of visits to each brand (rounded to the nearest integer)
#     df_grouped['frequency'] = df_grouped['score_sum'].round().astype(int)

#     # Calculate Tenure: time beteen the first and last visit
#     df_grouped['recency'] = (df_grouped['last_visit_date'] - df_grouped['first_visit_date']).dt.days

#     # Calculate Recency: time since the customer's last visit

#     # reference_date = df['DATE'].max()
#     # df_grouped['tenure'] = (reference_date - df_grouped['last_visit_date']).dt.days
#     df_grouped['tenure'] = 365

#     return df_grouped[['ID', 'BRAND', 'frequency', 'tenure', 'recency']]

In [31]:
train_X_grouped = calculate_rf_scores(train_X)
train_y_grouped = calculate_rf_scores(train_y)

test_X_grouped = calculate_rf_scores(test_X)
test_y_grouped = calculate_rf_scores(test_y)

predict_X_grouped = calculate_rf_scores(predict_X)

In [32]:
print("Train X grouped shape:", train_X_grouped.shape)
print("Train y grouped shape:", train_y_grouped.shape)
print("Test X grouped shape:", test_X_grouped.shape)
print("Test y grouped shape:", test_y_grouped.shape)
print("Predict X grouped shape:", predict_X_grouped.shape)

Train X grouped shape: (120005, 5)
Train y grouped shape: (80762, 5)
Test X grouped shape: (123992, 5)
Test y grouped shape: (81737, 5)
Predict X grouped shape: (119812, 5)


In [33]:
train_X_grouped.to_csv('Grouped/train_X_grouped.csv')
train_y_grouped.to_csv('Grouped/train_y_grouped.csv')

test_X_grouped.to_csv('Grouped/test_X_grouped.csv')
test_y_grouped.to_csv('Grouped/test_y_grouped.csv')

predict_X_grouped.to_csv('Grouped/predict_X_grouped.csv')

## Combine with Revenue

In [153]:
revenue_df = pd.read_csv('data_preprocessing/train_x_rfm.csv')
revenue_df = revenue_df[['ID', 'BRAND', 'RECENCY', 'FREQUENCY', 'REVENUE']]
revenue_df.head()

,ID,BRAND,RECENCY,FREQUENCY,REVENUE
0,000AHHU956,RetroReel,317,1,60.00
1,000DJKK225,CineQuest,260,1,48.00
2,000DTPC747,LimeLight,152,1,80.00
3,000ORNZ823,CineQuest,85,1,168.02
4,000QBES821,CineQuest,134,1,36.03


In [154]:
merged = pd.merge(train_X_grouped, revenue_df[['ID', 'BRAND', 'REVENUE']], how='outer', on=['ID', 'BRAND'])

# merged['recency'] = np.maximum(merged['recency'].fillna(0), merged['RECENCY'].fillna(0))
# merged['frequency'] = np.maximum(merged['frequency'].fillna(0), merged['FREQUENCY'].fillna(0))

# # Drop the redundant columns from df2
# merged = merged.drop(columns=['RECENCY', 'FREQUENCY'])

# # Fill 'tenure' and 'revenue' columns where they exist
# merged['tenure'] = merged['tenure'].fillna(0)
merged['monetary_value'] = merged['REVENUE'].fillna(0)
merged = merged.drop(columns=['REVENUE'])

merged.head()

,ID,BRAND,recency,frequency,tenure,monetary_value
0,000AHHU956,RetroReel,230.0,1.0,230.0,60.0
1,000AHHU956,SilverScreen Classics,156.0,1.0,156.0,0.0
2,000AHHU956,TimeCapsule TV,68.0,1.0,68.0,0.0
3,000AUEW012,Adrenaline,117.0,0.0,117.0,0.0
4,000AZAF724,Adrenaline,45.0,2.0,335.0,0.0


In [149]:
merged.shape

(121414, 5)

## Frequency/Recency analysis using the BG/NBD model

In [99]:
from lifetimes import BetaGeoFitter

In [160]:
df_brand_level = merged.groupby('BRAND').agg({
    'frequency': 'max',
    'recency': 'mean',
    'tenure': 'mean',
    'monetary_value': 'mean'
}).reset_index()

df_brand_level

,BRAND,frequency,recency,tenure,monetary_value
0,Adrenaline,5.0,148.911681,214.757530,0.000000
1,ChillStream,5.0,110.104681,253.220174,59.275827
2,CineQuest,5.0,110.802266,253.376191,45.893218
3,DarkMatter,4.0,147.467883,216.628315,0.000000
4,DesignLab,3.0,162.819202,201.132026,0.000000
5,LimeLight,5.0,129.683137,232.377443,34.585092
6,PopCulture Now,4.0,162.500175,200.409637,0.000000
7,Pulse,6.0,116.653807,248.619067,27.135842
8,RetroReel,6.0,116.894326,246.029078,40.962211
9,SilverScreen Classics,4.0,150.742252,214.243952,0.000000


In [161]:
# predict number of subscription coming from each brand

bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(df_brand_level['frequency'], df_brand_level['recency'], df_brand_level['tenure'])

t = 180 # 30 day period
df_brand_level['expected_sub_6_months'] = bgf.conditional_expected_number_of_purchases_up_to_time(t, df_brand_level['frequency'], df_brand_level['recency'], df_brand_level['tenure'])
df_brand_level.sort_values(by='expected_sub_6_months',ascending=False)

,BRAND,frequency,recency,tenure,monetary_value,expected_sub_6_months
6,PopCulture Now,4.0,162.500175,200.409637,0.000000,2.821431
4,DesignLab,3.0,162.819202,201.132026,0.000000,2.546293
10,TasteMakers,3.0,164.792460,202.705637,0.000000,2.546195
0,Adrenaline,5.0,148.911681,214.757530,0.000000,2.401815
9,SilverScreen Classics,4.0,150.742252,214.243952,0.000000,2.297469
11,TimeCapsule TV,4.0,148.434531,213.266609,0.000000,2.269473
3,DarkMatter,4.0,147.467883,216.628315,0.000000,2.170115
5,LimeLight,5.0,129.683137,232.377443,34.585092,1.395500
8,RetroReel,6.0,116.894326,246.029078,40.962211,0.746744
7,Pulse,6.0,116.653807,248.619067,27.135842,0.699201


In [171]:
merged

,ID,BRAND,recency,frequency,tenure,monetary_value
0,000AHHU956,RetroReel,230.0,1.0,230.0,60.0
1,000AHHU956,SilverScreen Classics,156.0,1.0,156.0,0.0
2,000AHHU956,TimeCapsule TV,68.0,1.0,68.0,0.0
3,000AUEW012,Adrenaline,117.0,0.0,117.0,0.0
4,000AZAF724,Adrenaline,45.0,2.0,335.0,0.0
...,...,...,...,...,...,...
121409,999ZGDS170,CineQuest,235.0,1.0,257.0,0.0
121410,999ZIAU985,RetroReel,54.0,1.0,347.0,0.0
121411,999ZTXB880,Pulse,41.0,2.0,300.0,0.0
121412,999ZYVM582,SilverScreen Classics,53.0,1.0,53.0,0.0


In [175]:
df_id_level = merged.groupby('ID').agg({
    'frequency': 'sum',                  # Total frequency across all brands
    'recency': 'min',                    # Minimum recency across all brands
    'tenure': 'max',                     # Maximum tenure across all brands
    'monetary_value': 'sum'              # Total monetary value across all brands
}).reset_index()

df_id_level.loc[df_id_level['frequency'] == 0, 'recency'] = 0

df_id_level

,ID,frequency,recency,tenure,monetary_value
0,000AHHU956,3.0,68.0,230.0,60.0
1,000AUEW012,0.0,0.0,117.0,0.0
2,000AZAF724,2.0,45.0,335.0,0.0
3,000BRHY264,1.0,142.0,317.0,0.0
4,000BRWF792,3.0,18.0,315.0,0.0
...,...,...,...,...,...
75436,999YJJT120,2.0,227.0,270.0,0.0
75437,999ZGDS170,1.0,235.0,257.0,0.0
75438,999ZIAU985,1.0,54.0,347.0,0.0
75439,999ZTXB880,2.0,41.0,300.0,0.0


In [179]:
# predict number of subscription coming from each customer

bgf = BetaGeoFitter(penalizer_coef=0.1)
bgf.fit(df_id_level['frequency'], df_id_level['recency'], df_id_level['tenure'])

t = 180 # 30 day period
df_id_level['expected_sub_6_months'] = bgf.conditional_expected_number_of_purchases_up_to_time(t, df_id_level['frequency'], df_id_level['recency'], df_id_level['tenure'])
df_id_level.sort_values(by='expected_sub_6_months',ascending=False)

  message: NaN result encountered.
  success: False
   status: 3
      fun: nan
        x: [ 1.000e-01  1.000e-01  1.000e-01  1.000e-01]
      nit: 0
      jac: [       nan        nan        nan        nan]
 hess_inv: [[1 0 0 0]
            [0 1 0 0]
            [0 0 1 0]
            [0 0 0 1]]
     nfev: 1
     njev: 1


ConvergenceError: 
The model did not converge. Try adding a larger penalizer to see if that helps convergence.


## Gamma-Gamma Model

In [ ]:
ggf = GammaGammaFitter(penalizer_coef=0.0)

ggf.fit(
    frequency = train_X_grouped['frequency'],
    monetary_value = df_grouped['monetary_value']
)

KeyError: 'monetary_value'

## Calculate CLV

In [ ]:
df_grouped['CLV'] = ggf.customer_lifetime_value(
    bgf,
    frequency = df_grouped['frequency'],
    recency = df_grouped['recency'],
    T = df_grouped['tenure'],
    monetary_value = df_grouped['monetary_value'],
    time = 6,  # months
    freq = 'D',  # 'D' for daily frequency
    discount_rate = 0.01  # Monthly discount rate (~12% annually)
)

2022-01-2022-12 train, 2022-06-01-2023-06-01 valid, 2023-01-2023-12 predict

y revenue 2023-01-2023-06 train; 2023-06-2023-12 valid, predict next 6 months